# Simulating low-light product inspection

This notebook shows how to systematically simulate a change in lighting conditions (i.e. a shift in data distribution), and the effect such a change has on model predictions. The notebook is using the SonomaCreek SDK to interact with SonomaCreek. 

**Problem description**: For any given project, a data scientist working on it may want to quantify the effect of image distortions on model performance. For example in medical imaging, there is always a certain noise background in the images depending on patient anatomy and radiation dose.

In this scenario the customer has set up an anomaly segmentation model to detect fabrication defects on discrete transistors mounted on a printed circuit board. For various reasons, the customer wants to reduce both the light intensity and exposure time in the inspection line. Let's assume the product they are inspecting is sensitive to light so they want to achieve the minimal exposure to light possible, but they still want to detect fabrication defects. How will this affect their anomaly segmentation model? Of course they will do real-world tests before implementing any changes, but is it possible to simulate such a shift in data distribution in advance? *With SonomaCreek, it is*.

**Project type**: Anomaly segmentation

**Project name**: Transistor anomaly segmentation


### Step 1: Connect to SonomaCreek

In [ ]:
from dotenv import dotenv_values
from sc_api_tools import SCRESTClient

env_variables = dotenv_values(dotenv_path="../.env")

# The SCRESTClient object establishes the connection to SonomaCreek
client = SCRESTClient(
    host=env_variables.get('HOST'),
    username=env_variables.get('USERNAME'),
    password=env_variables.get('PASSWORD')
)

### Step 2: Get project

In [ ]:
from sc_api_tools.rest_managers import ProjectManager

# The ProjectManager object allows getting, creating and 
# deleting projects from the SonomaCreek instance
project_manager = ProjectManager(session=client.session, workspace_id=client.workspace_id)

PROJECT_NAME = "Transistor anomaly segmentation"
project = project_manager.get_project_by_name(PROJECT_NAME)

print(project.summary)

### Step 3: Get images and annotations
Set up ImageManager for the project, get image metadata

In [ ]:
from sc_api_tools.rest_managers import ImageManager

image_manager = ImageManager(
    session=client.session, workspace_id=client.workspace_id, project=project
)

images = image_manager.get_all_images()
print(f"Project '{project.name}' contains {len(images)} images")

image_1 = images[0]

Set up AnnotationManager for the project, get annotations for first image in the project and inspect it.

In [ ]:
from sc_api_tools.rest_managers import AnnotationManager
from sc_api_tools.utils import show_image_with_annotation_scene

annotation_manager = AnnotationManager(
    session=client.session, workspace_id=client.workspace_id, project=project
)

annotation_1 = annotation_manager.get_annotation(image_1)

# Inspect annotation for image 1
image_1.get_data(client.session)
show_image_with_annotation_scene(image_1, annotation_1, show_in_notebook=True)

Get and inspect anomalous image with annotation

In [ ]:
image_2 = images[1]

annotation_2 = annotation_manager.get_annotation(image_2)

# Inspect annotation for image 2
image_2.get_data(client.session)
show_image_with_annotation_scene(image_2, annotation_2, show_in_notebook=True)

### Step 4: Get prediction for anomalous image
Set up prediction manager, fetch prediction for image

In [ ]:
from sc_api_tools.rest_managers import PredictionManager

prediction_manager = PredictionManager(session=client.session, workspace_id=client.workspace_id, project=project)

prediction = prediction_manager.get_image_prediction(image_2)

show_image_with_annotation_scene(image_2, prediction, show_in_notebook=True)

### Step 5: Simulate low light conditions
To simulate the reduced light intensity, we decrease the overall brightness and add shot noise to the image

In [ ]:
from image_utils import simulate_low_light_image, display_image_in_notebook

# Reduce brightness and add shot noise to simulate low light intensity and short exposure time
new_image_with_noise = simulate_low_light_image(image_2, reduction_factor=0.75)

display_image_in_notebook(new_image_with_noise)

### Step 6: Get prediction for the modified image

In [ ]:
sc_noisy_image, noisy_prediction = client.upload_and_predict_image(image=new_image_with_noise, project_name=PROJECT_NAME, visualise_output=False)
show_image_with_annotation_scene(new_image_with_noise, noisy_prediction, show_in_notebook=True)

### Step 7: Simulate a range of different light levels
Change the lighting reduction factor from very strong reduction (`reduction_factor=0.1`) to weak reduction (`reduction_factor=0.8`). 

In [ ]:
import numpy as np

start_factor = 0.1
stop_factor = 0.8
step = 0.1

for alpha in np.arange(start_factor, stop_factor, step):
    new_image_with_noise = simulate_low_light_image(
        image_2,
        reduction_factor=alpha
    )
    client.upload_and_predict_image(image=new_image_with_noise, project_name=PROJECT_NAME, visualise_output=False)

## Model re-training for low light conditions
Ok, so now we know that the *existing* model can still find anomalies, even in low light conditions. But, of course this is not a fair comparison since the low light images are not part of the training set for that model. Could we simulate training a completely new model on a low-light dataset? Yes we can!

### Step 8: Create a new project
Create a new project dedicated to images with a certain lighting reduction factor

In [ ]:
LIGHT_REDUCTION_FACTOR = 0.5

new_project = project_manager.get_or_create_project(
    project_name = PROJECT_NAME + f" light reduction factor {LIGHT_REDUCTION_FACTOR:.1f}",
    project_type = "anomaly_segmentation",
    labels=[[]]
)

### Step 9: Modify all images data and annotations

##### Loop over the existing images and:
  1. Create new image with simulated low light conditions
  2. Get existing annotation for image
  3. Apply existing annotation to simulated image

In [ ]:
new_image_manager = ImageManager(session=client.session, workspace_id=client.workspace_id, project=new_project)
new_annotation_manager = AnnotationManager(session=client.session, workspace_id=client.workspace_id, project=new_project)

for image in images:
    annotation = annotation_manager.get_annotation(image)
    numpy_image = image.get_data(client.session)
    new_image = simulate_low_light_image(
        numpy_image, 
        reduction_factor=LIGHT_REDUCTION_FACTOR
    )
    new_sc_image = new_image_manager.upload_image(new_image)
    new_annotation = annotation.map_labels(labels=new_project.get_all_labels())
    new_annotation_manager.upload_annotation(annotation_scene=new_annotation, media_item=new_sc_image)

### Step 10: Start training job and monitor progress

In [ ]:
from sc_api_tools.rest_managers import TrainingManager

training_manager = TrainingManager(session=client.session, workspace_id=client.workspace_id, project=new_project)

job = training_manager.train_task(task=0)

training_manager.monitor_jobs([job])

In [ ]:
# Cleaning up the first project by removing the uploaded images that don't have annotations

images_to_delete = []
for image in image_manager.get_all_images():
    if annotation_manager.get_annotation(image) is None:
        images_to_delete.append(image)
image_manager.delete_images(images_to_delete)